In [2]:
!pip install segmentation_models_pytorch

In [6]:
!pip install faiss-gpu

  Using cached faiss-gpu-1.7.1.post2.tar.gz (40 kB)
  Using cached faiss-gpu-1.7.1.post1.tar.gz (41 kB)
  Using cached faiss-gpu-1.7.1.tar.gz (40 kB)
  Using cached faiss-gpu-1.7.0.tar.gz (34 kB)
  Using cached faiss-gpu-1.6.5.tar.gz (28 kB)
  Using cached faiss-gpu-1.6.4.post2.tar.gz (25 kB)
  Using cached faiss-gpu-1.6.4.tar.gz (3.4 kB)
    ERROR: Command errored out with exit status 1:
     command: /Users/ashwindaswani/opt/anaconda3/bin/python -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/9d/w50qsl190hs4k58psvc494_40000gn/T/pip-install-hzcs2p21/faiss-gpu_7b13595832c84cffa003076b21c5fb1d/setup.py'"'"'; __file__='"'"'/private/var/folders/9d/w50qsl190hs4k58psvc494_40000gn/T/pip-install-hzcs2p21/faiss-gpu_7b13595832c84cffa003076b21c5fb1d/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'

In [10]:
!pip install torchutils

In [1]:
import segmentation_models_pytorch as smp
import torch, torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import Cityscapes
from torch.utils.data import Dataset
from pcs.utils import torchutils

from PIL import Image
import numpy as np
from torchvision import transforms
from pcs.utils.segmentation_helper import one_hot_masks, one_hot_mask_data_loader
# from pcs.models.head import CosineClassifier
import torchutils 


model = smp.Unet(classes=1, in_channels=3, encoder_name="resnet34", encoder_weights="imagenet")

dataset = Cityscapes(root = "./data", split='train', mode='fine', transform=transforms.ToTensor(), target_transform=transforms.ToTensor(), target_type="polygon")






In [2]:
dataset.targets[0]

['./data/gtFine/train/zurich/zurich_000069_000019_gtFine_color.png']

In [2]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [4]:
for i, (img, mask) in enumerate(dataloader):
    dick = np.load('labels.npy',allow_pickle='TRUE').item()
    new_mask=np.zeros((256, 256, 30))
    
    
    break
    

KeyError: (tensor([0.0275, 0.0549, 0.0863,  ..., 0.4784, 0.0000, 0.0000]), tensor([0.5255, 0.5098, 0.4980,  ..., 0.4588, 0.0000, 0.0000]), tensor([0.2902, 0.2902, 0.2902,  ..., 0.4353, 0.0314, 0.0000]), tensor([0.5569, 0.5490, 0.5373,  ..., 0.4157, 0.0706, 0.0000]), tensor([0.2980, 0.3059, 0.3137,  ..., 0.3961, 0.1059, 0.0000]), tensor([0.0000, 0.0000, 0.0000,  ..., 0.3765, 0.1373, 0.0000]), tensor([0.0000, 0.0000, 0.0196,  ..., 0.3608, 0.1647, 0.0000]), tensor([0.0000, 0.0000, 0.0000,  ..., 0.3451, 0.1922, 0.0000]), tensor([0.0000, 0.0000, 0.0157,  ..., 0.3294, 0.2157, 0.0000]), tensor([0.0000, 0.0000, 0.0000,  ..., 0.3176, 0.2392, 0.0000]), tensor([0.0000, 0.0000, 0.0000,  ..., 0.3098, 0.2588, 0.0000]), tensor([0.0000, 0.0000, 0.0000,  ..., 0.3020, 0.2784, 0.0000]), tensor([0.0000, 0.0000, 0.0000,  ..., 0.2980, 0.2941, 0.0000]), tensor([0.0000, 0.0000, 0.0000,  ..., 0.2941, 0.3098, 0.0000]), tensor([0.0000, 0.0000, 0.0000,  ..., 0.2941, 0.3255, 0.0000]), tensor([0.0000, 0.0000, 0.000

In [96]:
for i, (image, mask) in enumerate(dataloader):
    print(image.shape)
    print(mask.shape)
    break

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'PIL.PngImagePlugin.PngImageFile'>

In [3]:
def split_dataset():
    train_dataset = Cityscapes(root="./data", split="train", mode="fine", transform=transforms.ToTensor(), target_type="color")
    val_dataset = Cityscapes(root="./data", split="val", mode="fine", transform=transforms.ToTensor(), target_type="color")
    test_dataset = Cityscapes(root="./data", split="test", mode="fine", transform=transforms.ToTensor(), target_type="color")

In [9]:
train_dataset = Cityscapes(root="./data", split="train", mode="fine", transform=transforms.ToTensor(), target_type="color")
val_dataset = Cityscapes(root="./data", split="val", mode="fine", transform=transforms.ToTensor(), target_type="color")
test_dataset = Cityscapes(root="./data", split="test", mode="fine", transform=transforms.ToTensor(), target_type="color")

len(train_dataset), len(val_dataset), len(test_dataset)

(2975, 500, 1525)

In [2]:
import numpy as np
def create_label_index(dataset_size, percent_labeled=0.05):
    return np.random.choice(dataset_size, int(dataset_size * percent_labeled), replace=False)
    
create_label_index(100, 0.05)

array([81, 12, 21, 37, 70])

In [80]:
# open cityscapes/gtCoarse/train/ and gwt all the color.png files
import os
masks_path=[]
for root, dirs, files in os.walk("cityscapes/gtCoarse/test/"):
    for file in files:
        if file.endswith("color.png"):
             masks_path.append(os.path.join(root, file))
             print(os.path.join(root, file))

In [73]:
dick

{(0, 0, 0): 0,
 (70, 70, 70): 1,
 (180, 130, 70): 2,
 (0, 220, 220): 3,
 (35, 142, 107): 4,
 (153, 153, 153): 5,
 (30, 170, 250): 6,
 (70, 0, 0): 7,
 (142, 0, 0): 8,
 (0, 0, 255): 9,
 (32, 11, 119): 10,
 (232, 35, 244): 11,
 (128, 64, 128): 12,
 (60, 20, 220): 13,
 (156, 102, 102): 14,
 (153, 153, 190): 15,
 (100, 60, 0): 16,
 (230, 0, 0): 17,
 (152, 251, 152): 18,
 (100, 80, 0): 19,
 (160, 170, 250): 20,
 (110, 0, 0): 21,
 (140, 150, 230): 22,
 (180, 165, 180): 23,
 (81, 0, 81): 24,
 (90, 0, 0): 25,
 (100, 100, 150): 26,
 (90, 120, 150): 27}

In [79]:
count = 0
print(mask.shape)
#mask = cv2.resize(mask, (256, 256))
s=set()
cnt=29
for mask_path in masks_path:
    mask = cv2.imread(mask_path)
    mask = np.array(mask)
    for i in range(1024):
        for j in range(2048):
            if tuple(mask[i][j]) not in dick:
                dick[tuple(mask[i][j])]=cnt
                cnt+=1
                print(cnt)
            if cnt==29:
                break
print(len(s))

(1024, 2048, 3)
0


In [70]:
dick

{(0, 0, 0): 0,
 (70, 70, 70): 1,
 (180, 130, 70): 2,
 (0, 220, 220): 3,
 (35, 142, 107): 4,
 (153, 153, 153): 5,
 (30, 170, 250): 6,
 (70, 0, 0): 7,
 (142, 0, 0): 8,
 (0, 0, 255): 9,
 (32, 11, 119): 10,
 (232, 35, 244): 11,
 (128, 64, 128): 12,
 (60, 20, 220): 13,
 (156, 102, 102): 14,
 (153, 153, 190): 15,
 (100, 60, 0): 16,
 (230, 0, 0): 17,
 (152, 251, 152): 18,
 (100, 80, 0): 19,
 (160, 170, 250): 20,
 (110, 0, 0): 21,
 (140, 150, 230): 22,
 (180, 165, 180): 23,
 (81, 0, 81): 24,
 (90, 0, 0): 25,
 (100, 100, 150): 26,
 (90, 120, 150): 27}

In [77]:
# save dictionary in csv
import numpy as np

# Save

np.save('dick.npy', dick) 

# Load
# read_dictionary = np.load('my_file.npy',allow_pickle='TRUE').item()
# print(read_dictionary['hello']) # displays "world"

In [ ]:
for mask_path in masks_path:
    new_mask=np.zeros((1024, 2048, 30))
    for i in range(1024):
        for j in range(2048):
            mask = cv2.imread(mask_path)
            mask = np.array(mask)
            channel=dick[mask]
            new_mask[i][j][channel]= 1

In [69]:
import torch.nn as nn

img =torch.zeros(2, 3)
m = nn.MaxPool2d(3, stride=2)
#function to to downscale a 256x26x3 image to 8x8x3 using max pooing layer
def down_sample(img):
    down = m(img)
    print(down.size())

down_sample(img)



RuntimeError: non-empty 3D or 4D (batch mode) tensor expected for input

In [41]:
mask = "cityscapes/gtCoarse/train/aachen/aachen_000000_000019_gtCoarse_color.png"

mask = "/Users/ashwindaswani/Desktop/pcs/PCS-FUDA/cityscapes/gtCoarse/train/aachen/aachen_000000_000019_gtCoarse_instanceIds.png"
mask = "/Users/ashwindaswani/Desktop/pcs/PCS-FUDA/cityscapes/gtCoarse/train/aachen/aachen_000000_000019_gtCoarse_labelIds.png"
import cv2

def get_classes(mask):
    mask = cv2.imread(mask)
    mask = np.array(mask)
    print(mask.shape)
    mask = cv2.resize(mask, (256, 256))
    # mask.shape
    final=np.zeros((8,8,3))
    for i in range(8):
        for j in range(8):
            new_img = mask[i*32:(i+1)*32, j*32:(j+1)*32]
            # find the most frequent color
            final[i][j][0] = np.bincount(new_img[:,:,0].flatten()).argmax()
            final[i][j][1] = np.bincount(new_img[:,:,1].flatten()).argmax()
            final[i][j][2] = np.bincount(new_img[:,:,2].flatten()).argmax()
            
    cv2.imwrite("final.png", final)
    
    return final
o = get_classes(mask)
print(o.shape)

(1024, 2048, 3)
(8, 8, 3)


In [144]:
X = (8,8,512)
y = (8,8,3)

In [148]:
img = dataset[1][1]

img = np.array(img)
print(img.shape)

(1024, 2048, 4)


In [3]:
encoded_X =[]
for data in dataset:
    X, y = data
    X = X.reshape(1,3,1024,2048)
    
    encoded = model.encoder(X)
    encoded_X.append(encoded[-1])
    y = get_classes(y)
    # y = y.reshape(1,8,8,3)

: 

: 

In [104]:
# show final image
final_img = Image.fromarray(final.astype('uint8'), 'RGB')

In [107]:
final_img.show()

In [70]:
# forwards pass only on encoder
# model = smp.Unet(classes=1, in_channels=3, encoder_name="resnet34", encoder_weights="imagenet")
out = model.encoder(torch.from_numpy(x).unsqueeze(0).float())

In [74]:
out[1].shape


torch.Size([1, 64, 128, 128])

In [76]:
for i in range(len(encoded)):
    print(encoded[i].shape)

torch.Size([1, 3, 256, 256])
torch.Size([1, 64, 128, 128])
torch.Size([1, 64, 64, 64])
torch.Size([1, 128, 32, 32])
torch.Size([1, 256, 16, 16])
torch.Size([1, 512, 8, 8])


In [77]:
out = model.encoder(torch.from_numpy(x).unsqueeze(0).float())
out = out[-1]


In [79]:
out.shape

torch.Size([1, 512, 8, 8])

In [82]:
dataset = "cityscapes/gtCoarse/train/aachen"
import os
data = []
for file in os.listdir(dataset):
    if file.endswith(".png") and file.__contains__("color"):
        data.append(file)


In [16]:
dataset = load_dataset("Chris1/GTA5", split="train")

# Get the first 1000 images
images = dataset['train']['image'][:1000]

# Save the images to a directory
for i, image in enumerate(images):
    with open(f'image_{i}.png', 'wb') as f:
        f.write(image)

KeyboardInterrupt: 

In [83]:
len(data)


174

In [84]:
# json_path ="cityscapes/gtCoarse/train/aachen/aachen_000000_000019_gtCoarse_polygons.json"

import json
json_path ="cityscapes/gtCoarse/train/aachen/aachen_000000_000019_gtCoarse_polygons.json"
target = json.load(open(json_path))

In [85]:
mask

{'imgHeight': 1024,
 'imgWidth': 2048,
 'objects': [{'label': 'traffic sign',
   'polygon': [[104, 265], [47, 354], [161, 363], [142, 315], [104, 276]]},
  {'label': 'traffic sign',
   'polygon': [[112, 390], [64, 476], [174, 466], [128, 414]]},
  {'label': 'terrain',
   'polygon': [[13, 582],
    [13, 754],
    [233, 675],
    [322, 593],
    [320, 580],
    [300, 573],
    [278, 558],
    [247, 549],
    [226, 533],
    [177, 521],
    [151, 524],
    [142, 533],
    [143, 606],
    [136, 615],
    [117, 623],
    [104, 623],
    [96, 611],
    [100, 565],
    [107, 546],
    [65, 519],
    [57, 519],
    [51, 525],
    [40, 574],
    [25, 585]]},
  {'label': 'vegetation',
   'polygon': [[358, 236],
    [341, 249],
    [305, 302],
    [300, 335],
    [306, 342],
    [340, 349],
    [380, 350],
    [412, 362],
    [441, 364],
    [478, 345],
    [520, 341],
    [529, 328],
    [533, 321],
    [530, 308],
    [497, 282],
    [482, 279],
    [442, 263],
    [407, 244]]},
  {'label': 'ca

In [2]:
!pip install dotmap

In [3]:

import torch
import torch.nn as nn
import torch.nn.functional as F
from pcs.utils.torchutils import grad_reverse
from torch.autograd import Function
from torchvision import models
class CosineClassifier(nn.Module):
    def __init__(self, num_class=64, inc=4096, temp=0.05):
        super(CosineClassifier, self).__init__()
        self.fc = nn.Linear(inc, num_class, bias=False)
        self.num_class = num_class
        self.temp = temp

    def forward(self, x, reverse=False, eta=0.1):
        self.normalize_fc()

        if reverse:
            x = grad_reverse(x, eta)
        x = F.normalize(x)
        x_out = self.fc(x)
        x_out = x_out / self.temp

        return x_out

    def normalize_fc(self):
        self.fc.weight.data = F.normalize(self.fc.weight.data, p=2, eps=1e-12, dim=1)

    @torch.no_grad()
    def compute_discrepancy(self):
        self.normalize_fc()
        W = self.fc.weight.data
        D = torch.mm(W, W.transpose(0, 1))
        D_mask = 1 - torch.eye(self.num_class).cuda()
        return torch.sum(D * D_mask).item()

In [19]:

def weights_init(model):
    for layer in model.modules():
        if isinstance(layer, torch.nn.Conv2d):
            torch.nn.init.kaiming_normal_(
                layer.weight, mode="fan_out", nonlinearity="relu"
            )
            if layer.bias is not None:
                torch.nn.init.constant_(layer.bias, val=0.0)
        elif isinstance(layer, torch.nn.BatchNorm2d):
            torch.nn.init.constant_(layer.weight, val=1.0)
            torch.nn.init.constant_(layer.bias, val=0.0)
        elif isinstance(layer, torch.nn.Linear):
            torch.nn.init.xavier_normal_(layer.weight)
            if layer.bias is not None:
                torch.nn.init.constant_(layer.bias, val=0.0)

In [29]:
def convert_image_to_regions(feature):
    batch_size, img_embedding_rows, img_embedding_cols, img_embedding_dim = feature.shape
    regions = feature.reshape(batch_size* img_embedding_rows* img_embedding_cols, img_embedding_dim)
    return regions

fearuer = torch.randn(10,  8, 8, 512)
print(convert_image_to_regions(fearuer).shape)



torch.Size([640, 512])


In [27]:
def initialize_centroids(feat, batch_size, klist, cosine_classifier):
    lbd = cosine_classifier(feat)
    #lbd = lbd.reshape(batch_size, 64, 30)
    assert lbd.shape == (batch_size, 64, 30) 
    lbd = lbd.reshape(batch_size, 64, 30)
    lbd = lbd.reshape(batch_size*64, 30)
    
    masks = lbd
    embeddings = feat.reshape(batch_size* 64, 512)
    
    unique_labels = torch.unique(lbd, dim=0)
    keys = np.arange(30)
    
        
    
    indices = [torch.nonzero(masks[:, i], as_tuple=False) for i in range(masks.shape[1])]

    embeddings_list = [embeddings[indices[i]] for i in range(len(indices))]
  
    embeddings_list = [embeddings_list[i].float() for i in range(len(embeddings_list))]
    mean_embeddings = []
    mean_embeddings = [torch.mean(embeddings_list[i], dim=0) for i in range(len(embeddings_list))]
    
    
    # print(mean_embeddings)
    #dictionary_indices = dict(zip(keys, indices))
    dictionary_mean = dict(zip(keys, mean_embeddings))
    if len(unique_labels) < 30:
    
        for i in range(30):
            if i not in unique_labels:
                #assign random embedding
                dictionary_mean[i] = torch.rand(512)
    nparray = np.array(list(dictionary_mean.values()))
    return nparray

feat = torch.rand(1, 64, 512)
cls_head = CosineClassifier(
    num_class=30, inc=512, temp=0.05)

weights_init(cls_head)
dictio = initialize_centroids(feat, 1, 30, cls_head)
dictio[0].shape

torch.Size([1, 512])

In [5]:
torch.rand(3)

tensor([0.9283, 0.2287, 0.6134])